<a href="https://colab.research.google.com/github/Aroobmushtaq/Ai/blob/main/06_agents/Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai python-dotenv pydantic


In [ ]:
from openai import OpenAI

# 🔑 Paste your Gemini API key directly here
GEMINI_API_KEY = "past api key here or in secret"

# Initialize the OpenAI client with Gemini base URL
client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Ask user input
query = input("Enter your question: ")

# Send query to Gemini model
response = client.chat.completions.create(
    model="gemini-2.0-flash",   # You can try gemini-1.5-flash if this fails
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": query}
    ]
)

# Print output
print("\nAssistant:", response.choices[0].message.content)


Enter your question: weathe of karachi

Assistant: I can provide you with the current weather conditions and a short-term forecast for Karachi. However, since weather changes rapidly, I highly recommend checking a reliable weather app or website for the very latest information.

**To give you the most helpful answer, I need to know what kind of weather information you're looking for.** For example, are you interested in:

*   **The current temperature?**
*   **The overall conditions (sunny, cloudy, rainy, etc.)?**
*   **The wind speed and direction?**
*   **The humidity?**
*   **A forecast for today or the next few days?**

**In the meantime, here's some general information about Karachi's weather:**

*   Karachi has a hot, semi-arid climate.
*   Summers (roughly March to November) are hot and humid.
*   Winters (roughly December to February) are mild and dry.
*   The monsoon season brings occasional rainfall.

Please tell me what specific weather information you'd like, and I'll do my

**Practice with Context**

Context is just a container (or a box) that holds the data the agent needs while it’s running.
For example, the context might store things like:

* The user’s name or ID

* Settings or preferences

* Extra data the agent needs to do its job

In [ ]:
import os
from dataclasses import dataclass
from dotenv import load_dotenv

# --- Load or set API key ---
os.environ["GEMINI_API_KEY"] = "key here"
gemini_api_key = os.getenv("GEMINI_API_KEY")

# --- Import OpenAI client ---
from openai import AsyncOpenAI

# --- Mock classes (to simulate missing ones) ---
class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Agent:
    def __init__(self, name, instructions, model, tools):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.tools = tools

class Runner:
    @staticmethod
    def run_sync(agent, query, context):
        for tool in agent.tools:
            info = tool(context)
            return type("Result", (), {"final_output": f"Query: {query}\n{info}"})

def function_tool(func):
    return func

class RunContextWrapper:
    def __init__(self, context):
        self.context = context

# --- Create AsyncOpenAI client ---
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# --- User class ---
@dataclass
class User:
    user_id: int

# ✅ FIXED HERE: remove `[User]` — just use RunContextWrapper normally
@function_tool
def get_user_info(ctx: RunContextWrapper) -> str:
    """Fetches user personal information."""
    id = ctx.context.user_id
    if id == 1:
        return "User name is Ali. He is 19 years old. He is an Agentic AI Engineer who likes playing Cricket."
    elif id == 2:
        return "User name is Usman. He is 30 years old. He is a doctor who likes mountains."
    else:
        return "User not found"

# --- Create Agent ---
agent = Agent(
    name="Assistant",
    instructions="You are an expert in agentic AI.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    tools=[get_user_info]
)

# --- Run simulation ---
query = input("Enter the query: ")

result = Runner.run_sync(
    agent,
    query,
    context=RunContextWrapper(User(user_id=1))
)

print(result.final_output)


Enter the query: What is the user info?
Query: What is the user info?
User name is Ali. He is 19 years old. He is an Agentic AI Engineer who likes playing Cricket.


*Practice for output type in agents*

In [ ]:
# ------------------------- CREATE agents.py FILE -------------------------
import textwrap

code = textwrap.dedent("""
class Agent:
    def __init__(self, name, instructions, model):
        self.name = name
        self.instructions = instructions
        self.model = model

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    def run_sync(agent, query):
        response = agent.model.client.chat.completions.create(
            model=agent.model.model,
            messages=[
                {"role": "system", "content": agent.instructions},
                {"role": "user", "content": query}
            ]
        )
        return type("Result", (), {"final_output": response.choices[0].message.content})
""")

with open("agents.py", "w") as f:
    f.write(code)

# ------------------------- MAIN SCRIPT -------------------------
from openai import OpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner
from pydantic import BaseModel
from typing import List

# Hard-coded Gemini API key ⚠️ (replace with your actual key)
GEMINI_API_KEY = "your key here"

# Create Gemini client (sync version)
client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Define data model for quiz
class Quiz(BaseModel):
    question: str
    options: List[str]
    correct_option: str

# Create agent
agent = Agent(
    name="Assistant",
    instructions="You are a Quiz Agent. You generate quizzes with question, 4 options, and correct answer.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    output_type=Quiz
)

# Run in Colab
query = input("Enter your quiz topic: ")

result = Runner.run_sync(agent, query)

print("\n--- Quiz Generated ---\n")
print(result.final_output)


<frozen posixpath>:82: RuntimeWarning: coroutine 'AsyncCompletions.create' was never awaited


Enter your quiz topic: html

--- Quiz Generated ---

Okay, I will generate a quiz about HTML.

**Question 1:**

Which of the following is the correct way to create a hyperlink in HTML?

(A)  `<link url="http://www.example.com">Example</link>`
(B)  `<a href="http://www.example.com">Example</a>`
(C)  `<url>http://www.example.com</url>Example`
(D)  `<hyperlink>http://www.example.com</hyperlink>Example`

**Correct Answer:** (B)

**Question 2:**

Which HTML tag is used to define an internal style sheet?

(A) `<style>`
(B) `<css>`
(C) `<script>`
(D) `<link>`

**Correct Answer:** (A)

**Question 3:**

Which of the following HTML tags is used to display images?

(A) `<image src="image.gif">`
(B) `<img src="image.gif">`
(C) `<picture src="image.gif">`
(D) `<icon src="image.gif">`

**Correct Answer:** (B)

**Question 4:**

What does HTML stand for?

(A) Hyper Text Markup Language
(B) Hyperlinks and Text Markup Language
(C) Home Tool Markup Language
(D) Highly Typed Machine Language

**Correct An

**Handoffs**

Handoffs are sub‑agents the agent can delegate to. When a handoff occurs, the delegated agent receives the conversation history and takes over the conversation. This pattern enables modular, specialized agents that excel at a single task. Read more in the handoffs documentation.

In [1]:
# ------------------------ CREATE agents.py FILE ------------------------
import textwrap

code = textwrap.dedent("""
class Agent:
    def __init__(self, name, instructions, model, tools=None, handoffs=None):
        self.name = name
        self.instructions = instructions
        self.model = model
        self.tools = tools or []
        self.handoffs = handoffs or []

class OpenAIChatCompletionsModel:
    def __init__(self, model, openai_client):
        self.model = model
        self.client = openai_client

class Runner:
    @staticmethod
    def run_sync(agent, query):
        response = agent.model.client.chat.completions.create(
            model=agent.model.model,
            messages=[
                {"role": "system", "content": agent.instructions},
                {"role": "user", "content": query}
            ]
        )
        return type("Result", (), {"final_output": response.choices[0].message.content})

def function_tool(func):
    return func
""")

with open("agents.py", "w") as f:
    f.write(code)

# ------------------------ MAIN SCRIPT ------------------------
from openai import OpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool

# ⚠️ Hardcode your Gemini API key here
GEMINI_API_KEY = "your key here"

# Create client
client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Define tools
@function_tool
def get_refund(amount) -> str:
    print(f"Fetching refund status for {amount}...")
    return "Refund approved"

@function_tool
def book_flight(location) -> str:
    print(f"Booking flight for {location}...")
    return "Flight booked successfully."

# Define agents
booking_agent = Agent(
    name="Booking agent",
    instructions="You are a booking agent.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    tools=[book_flight],
)

refund_agent = Agent(
    name="Refund agent",
    instructions="You are a refund agent.",
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    tools=[get_refund],
)

triage_agent = Agent(
    name="Triage agent",
    instructions=(
        "Help the user with their questions. "
        "If they ask about booking, handoff to the booking agent. "
        "If they ask about refunds, handoff to the refund agent."
    ),
    model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client),
    handoffs=[booking_agent, refund_agent],
)

# Instead of input(), just assign your query directly
query = "I want to book a flight to Karachi"

# Run
result = Runner.run_sync(triage_agent, query)

print("\\n--- Final Output ---\\n")
print(result.final_output)


\n--- Final Output ---\n
```tool_code
handoff_to_agent(agent_name="booking_agent")
```
